# Docker DB서버와 연동

In [222]:
import cx_Oracle
import pandas as pd
import json
from pyarrow import csv
import numpy as np

In [17]:
data_path ='../data/'
# Json 파일 읽어오기
def read_json(jsonPath ,mod = 'r',encoding ='utf-8'):
    with open(jsonPath, mod, encoding = encoding) as common:
        config = json.load(common)
    return config
config = read_json(data_path + "json/config.json")

## Oracle 접속

In [7]:
CONN_INFO = {
        'NAME': 'XEPDB1',
        'USER': 'iitp',
        'PASSWORD': 'iitp',
        'HOST': '172.16.30.241',
        'PORT': '11521',
}
CONN_STR = '{USER}/{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**CONN_INFO)
conn = cx_Oracle.connect(CONN_STR)
cursor = conn.cursor()

# 필요 Table Create 및 

## 명세 테이블 생성 및 삽입

In [75]:
# 시도 코드 명세 테이블 생성
try :
    cursor.execute("""
        DROP TABLE sido_code 
    """)
except:
    pass    
finally:   
    cursor.execute("""
        CREATE TABLE sido_code ( 
            code    NUMBER(4)	NOT NULL,
            eng_name    VARCHAR2(20) NOT NULL, 
            kor_name    VARCHAR2(6) primary key
        )
    """) 
    # 값 삽입
    df = csv.read_csv(config['data_path']+ 'csv/specification/sido_code.csv').to_pandas()
    sql = """insert into sido_code(code,eng_name,kor_name)
             values (:1,:2,:3)"""         
    val = list(map(tuple,df.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()

In [156]:
# 범죄 명세 테이블 생성

try :
    cursor.execute("""
        DROP TABLE crime_Specification 
    """)
except:
    pass
        
finally:
    cursor.execute("""
        CREATE TABLE crime_Specification ( 
            code    NUMBER(4)	NOT NULL,
            crime_name    VARCHAR2(30) primary key
        )
    """)
    # 값 삽입
    df = csv.read_csv(config['data_path']+ 'csv/specification/crime_Specification.csv').to_pandas()
    sql = """insert into crime_Specification(code,crime_name)
             values (:1,:2)"""         
    val = list(map(tuple,df.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()

## 정제된 데이터 테이블 생성 및 삽입

In [274]:
# 시도 코드 맵핑
sido = pd.read_sql("select * from sido_code",con=conn) 
merged_csv = csv.read_csv(config['data_path']+ 'csv/cleaned/crime_indexed.csv').to_pandas()
merged_csv['시도'] = merged_csv['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])
crime = merged_csv.iloc[:,:4]
crime = crime.astype(str).fillna('')

In [278]:
# 범죄 명세 테이블 생성

try :
    cursor.execute("""
        DROP TABLE crime 
    """)
except:
    pass        
finally:
    cursor.execute("""
        create table crime ( 
            시도    varchar2(38) ,
            년도    varchar2(38) ,
            범죄종류    varchar2(38),
            범죄수    varchar2(38)
        )
    """)    
    sql = """insert into crime(시도 ,년도 ,범죄종류 ,범죄수)
             values (:1,:2,:3,:4)"""     
    val = list(map(tuple,crime.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    

In [280]:
independent_val = [
    '시도',
    '년도',
    '비경제활동인구',
    '취업자',
    '고용률 (%)',
    '실업자',
    '실업률 (%)',
    '1인당 지역내총생산',
    '1인당 지역총소득',
    '1인당 개인소득',
    '1인당 민간소비',
    'PC방',
    '백화점',
    '음식점',
    '유흥가',
    '향정신성의약품 청구건수',
    '경찰청 소속 경찰관 수',
    '경찰청 인원 1명당 담당 인구',
    '한국인(총인구수[명])',
    '한국인(남녀비율[백분율])',
    '한국인(인구밀도)',
    '총전입 (명)',
    '총전출 (명)',
    '대학교 수',
    '종교단체수',
    '외국인수',
    '총인구수 (명)']
indi_val = merged_csv[independent_val]


In [290]:
# 범죄 명세 테이블 생성

try :
    cursor.execute("""
        DROP TABLE independent_val 
    """)
except:
    pass        
finally:
    cursor.execute("""
        create table independent_val ( 
            시도 varchar2(38),
            년도 varchar2(38),
            비경제활동인구 varchar2(38),
            취업자 varchar2(38),
            고용률 varchar2(38),
            실업자 varchar2(38),
            실업률 varchar2(38),
            인당_지역내총생산 varchar2(38),
            인당_지역총소득 varchar2(38),
            인당_개인소득 varchar2(38),
            인당_민간소비 varchar2(38),
            PC방 varchar2(38),
            백화점 varchar2(38),
            음식점 varchar2(38),
            유흥가 varchar2(38),
            향정신성의약품_청구건수 varchar2(38),
            경찰청_소속_경찰관_수 varchar2(38),
            경찰청_인원_1명당_담당_인구 varchar2(38),
            한국인 varchar2(38),
            한국인비율 varchar2(38),
            한국인_인구밀도 varchar2(38),
            총전입 varchar2(38),
            총전출 varchar2(38),
            대학교수 varchar2(38),
            종교단체수 varchar2(38),
            외국인수 varchar2(38),
            총인구수 varchar2(38)
        )
    """)
    
    sql = """insert into independent_val(시도,년도,비경제활동인구,취업자,고용률,실업자,실업률,인당_지역내총생산,인당_지역총소득,인당_개인소득,인당_민간소비,PC방,백화점,음식점,유흥가,향정신성의약품_청구건수,경찰청_소속_경찰관_수,경찰청_인원_1명당_담당_인구,한국인,한국인비율,한국인_인구밀도,총전입,총전출,대학교수,종교단체수,외국인수,총인구수)
             values (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27)"""
    indi_val = indi_val.astype(str).fillna('')
    indi_val = indi_val.drop_duplicates()
    val = list(map(tuple,indi_val.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    

In [292]:
import cx_Oracle
import pandas as pd
CONN_INFO = {
        'NAME': 'XEPDB1',
        'USER': 'iitp',
        'PASSWORD': 'iitp',
        'HOST': '172.16.30.241',
        'PORT': '11521',
}
CONN_STR = '{USER}/{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**CONN_INFO)
conn = cx_Oracle.connect(CONN_STR)
cursor = conn.cursor()

sido_code = pd.read_sql("select * from sido_code",con=conn) 
crime_code =  pd.read_sql("select * from crime_Specification",con=conn) 
indi_val = pd.read_sql("select * from independent_val",con=conn) 
crime_val = pd.read_sql("select * from crime",con=conn) 

In [297]:
sido_code

,CODE,ENG_NAME,KOR_NAME
0,42,Gangwon-do,강원
1,41,Gyeonggi-do,경기
2,48,Gyeongsangnam-do,경남
3,47,Gyeongsangbuk-do,경북
4,29,Gwangju,광주
5,27,Daegu,대구
6,30,Daejeon,대전
7,26,Busan,부산
8,11,Seoul,서울
9,36,Sejong-si,세종
